# Interactions

In [1]:
import sys
import glob
import numpy as np
import pandas as pd
from collections import *

sys.path.append('/home/kalkiek/projects/reddit-political-affiliation/')

from src.data.date_helper import read_submissions
from src.features.interactions.political_comment import PoliticalComment
from transformers import BertTokenizer,BertForSequenceClassification, BertConfig, AdamW
import torch
import torch.optim as optim
import torch.nn as nn

wandb: WARNING W&B installed but not logged in.  Run `wandb login` or set the WANDB_API_KEY env variable.


### Quick Clean Up

I forgot to  remove 't1' prefix from some of the comment IDs

We also need to remove any bot accounts

In [2]:
def grab_bot_accounts():
    fname = '/shared/0/projects/prosocial/known-bots.tsv'
    bots = []

    with open(fname, 'rt') as f:
        lines = f.readlines()

        for line in lines:
            bots.append(line.split('\t')[1])

    print("Known bots: %d" % len(bots))
    return bots

bots = grab_bot_accounts()
bots.extend('[deleted]')

Known bots: 393


###  Can't run this until the script is done running! We'll run it in a few days

In [6]:
# def remove_id_prefix_and_bots(in_file):
#     comments = 0
#     with open(in_file, 'r') as f:
#         for line in f:
#             comment_id, parent_id, username, subreddit, created, politics, text = line.split('\t')
#             if username not in bots:
#                 if comment_id[:2] == 't1':
#                     comment_id = comment_id[3:]
#                 if parent_id[:2] == 't1':
#                     parent_id = parent_id[3:]

#                 comments += 1
#                 political_comment = PoliticalComment(comment_id, parent_id, username, subreddit, created, politics, text)
#                 yield political_comment

#     print("Total number of political comments: {}".format(comments))

# in_file = '/shared/0/projects/reddit-political-affiliation/data/interactions/all_comments.tsv'
# out_file = '/shared/0/projects/reddit-political-affiliation/data/interactions/all_comments_filtered.tsv'

# with open(out_file, 'w') as out_f:
#     for comment in filtered_comments(in_file):
#         out_f.write(comment.to_tsv_row())

Total number of political comments: 113921968


## Reading In The Data

Sample of how to read the data in

In [3]:
in_file = '/shared/0/projects/reddit-political-affiliation/data/interactions/all_comments_filtered.tsv'

def read_in_comments(in_file, count=-1):
    comments = []
    with open(in_file, 'r',encoding="utf-8") as f:
        for line in f:
            line = line.strip()
            try:
                comment_id, parent_id, username, subreddit, created, politics, text = line.split('\t')
                political_comment = PoliticalComment(comment_id, parent_id, username, subreddit, created, politics, text)
                comments.append(political_comment.to_dict())
                if count > 0 and len(comments) >= count:
                    return comments
            except Exception:
                pass
            
    print("Total number of political comments: {}".format(len(comments)))
    return comments


        
comments = read_in_comments(in_file, count=10000000)
df_comments = pd.DataFrame(comments)
print(len(df_comments))
df_comments.head(10)

10000000


,comment_id,parent_id,username,subreddit,created,politics,text
0,eypc9zv,eyp9duh,CueDramaticMusic,tumblr,1567296002,Democrat,I don’t know whether to read the last sentence...
1,eypca5l,eypby8b,ninethirtyone,news,1567296005,Democrat,You responded to my comment to make a point yo...
2,eypca5t,t3_cy2nlj,lostapwbm,SquaredCircle,1567296006,Democrat,After the fantastic energy of the crowds for T...
3,eypca8l,eypbz6d,OlliesFreeOxen,news,1567296007,Unknown,Run away when you lose. Typical. You have your...
4,eypcaac,eyp5fjl,Based_Putin,news,1567296008,Unknown,What does this even mean?
5,eypcabv,eyonlbn,needler14,ABoringDystopia,1567296009,Democrat,Right? I'll eat McDonald's here and there but ...
6,eypcac1,t3_cy09os,LimpWibbler_,funny,1567296009,Democrat,I absolutely love these kids. Funny or not.
7,eypcacv,t3_cy2yrm,DonnyGoat,NYYankees,1567296010,Democrat,Whose match are they at?
8,eypcae0,eyp66ij,rhunter99,news,1567296010,Unknown,Neil is that you?
9,eypcaeh,eypavwe,Firtydilthythrowaway,Music,1567296011,Unknown,This place is lost; it's merely a public opini...


# Bucket about dyadic affiliation composition (e.g,. Liberal+Liberal)

In [4]:
rep_df = df_comments[df_comments['politics'] == 'Republican']
dem_df = df_comments[df_comments['politics'] == 'Democrat']
unknown_df = df_comments[df_comments['politics'] == 'Unknown']

rep_users = set(rep_df['username'].tolist())
dem_users = set(dem_df['username'].tolist())
unknown_users = set(unknown_df['username'].tolist())

print("Number of rep users: {}".format(len(rep_users)))
print("Number of dem users: {}".format(len(dem_users)))
print("Number of unknown users: {}".format(len(unknown_users)))
print()
print("Number of rep comments: {}".format(len(rep_df)))
print("Number of dem comments: {}".format(len(dem_df)))
print("Number of unknown comments: {}".format(len(unknown_df)))

Number of rep users: 9133
Number of dem users: 19280
Number of unknown users: 845189

Number of rep comments: 2471970
Number of dem comments: 5243945
Number of unknown comments: 2284085


In [5]:
def get_interactions(from_party, to_party):
    from_comment_ids = set(df_comments[df_comments['politics'] == from_party]['comment_id'].tolist())
    to_comment_ids = set(df_comments[df_comments['politics'] == to_party]['comment_id'].tolist())
    interactions = df_comments[(df_comments['comment_id'].isin(from_comment_ids) & df_comments['parent_id'].isin(to_comment_ids))]
    return interactions

dem_to_dem = get_interactions('Democrat', 'Democrat')
rep_to_rep = get_interactions('Republican', 'Republican')
dem_to_rep = get_interactions('Democrat', 'Republican')
rep_to_dem = get_interactions('Republican', 'Democrat')
dem_to_unknown = get_interactions('Democrat', 'Unknown')
rep_to_unknown = get_interactions('Republican', 'Unknown')
unknown_to_dem = get_interactions('Unknown','Democrat')
unknown_to_rep = get_interactions('Unknown','Republican')

print("Dem to dem interactions: {}".format(len(dem_to_dem)))
print("Rep to rep interactions: {}".format(len(rep_to_rep)))
print("Dem to rep interactions: {}".format(len(dem_to_rep)))
print("Rep to dem interactions: {}".format(len(rep_to_dem)))
print("Dem to unknown interactions: {}".format(len(dem_to_unknown)))
print("Rep to unknown interactions: {}".format(len(rep_to_unknown)))
print("Unknown to dem interactions: {}".format(len(unknown_to_dem)))
print("Unknown to rep interactions: {}".format(len(unknown_to_rep)))


Dem to dem interactions: 116893
Rep to rep interactions: 29623
Dem to rep interactions: 49723
Rep to dem interactions: 48574
Dem to unknown interactions: 312114
Rep to unknown interactions: 145044
Unknown to dem interactions: 1408713
Unknown to rep interactions: 637680


In [6]:
comment_lists={"d2d":dem_to_dem,"r2r":rep_to_rep,"d2r":dem_to_rep,"r2d":rep_to_dem,"d2u":dem_to_unknown,"r2u":rep_to_unknown,"u2d":unknown_to_dem,"u2r":unknown_to_rep}

# Comments in same subreddit made at a similar time point

In [7]:
def prepare_features(seq_1, max_seq_length=128, zero_pad=True, include_CLS_token=True, include_SEP_token=True):
    # Tokenize Input
    tokens = tokenizer.tokenize(seq_1)[0:max_seq_length - 2]
    # Initialize Tokens
    if include_CLS_token:
        tokens.insert(0, tokenizer.cls_token)

    if include_SEP_token:
        tokens.append(tokenizer.sep_token)

    input_ids = tokenizer.convert_tokens_to_ids(tokens)
    pad_id = tokenizer.convert_tokens_to_ids([tokenizer.pad_token])[0]
    # Input Mask
    input_mask = [1] * len(input_ids)
    # Zero-pad sequence length
    if zero_pad:
        while len(input_ids) < max_seq_length:
            input_ids.append(pad_id)
            input_mask.append(0)
    # print(torch.tensor(input_ids).shape)
    return torch.tensor(input_ids), torch.tensor(input_mask)

In [8]:
config = BertConfig.from_pretrained('bert-base-uncased')
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForSequenceClassification(config)
dv='cuda:2'
model.load_state_dict(torch.load("best_bert.pt",map_location=torch.device(dv)))
device=torch.device(dv)
torch.cuda.set_device(int(dv[-1]))
model.cuda()
model.eval()

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

In [9]:
def counting_offensiveness_by_interactions(interactions,limit=3000):
    Counting=Counter()#{'Republican':Counter(),'Democrat':Counter(),'Unknown':Counter()}
    for idx,(comment_id,line) in enumerate(interactions.iterrows()):
        #print(line)
        text_id,mask=prepare_features(line['text'])
        text_id=text_id.unsqueeze(0).cuda()
        mask=mask.unsqueeze(0).cuda()

        output = model.forward(text_id,attention_mask=mask)[0]
        _, predicted = torch.max(output.data, 1)
        Counting[predicted.item()]+=1
#         if line['subreddit'] in political_subreddit:
#             Politics_Counting[line['politics']][predicted.item()]+=1
#         #print(idx)
        if idx==limit:
            break
    return Counting#,Politics_Counting
        

In [113]:
all_Counting=counting_offensiveness_by_interactions(df_comments,20000)

In [111]:
counting_list={}
for key in comment_lists:
    print(key)
    counting_list[key]=counting_offensiveness_by_interactions(comment_lists[key])

d2d
r2r
d2r
r2d
d2u
r2u
u2d
u2r


In [116]:
for key in counting_list:
    print(key,counting_list[key][1]/(counting_list[key][1]+counting_list[key][0]))
print("All",all_Counting[1]/(all_Counting[1]+all_Counting[0]))

d2d 0.32122625791402865
r2r 0.32090077410274453
d2r 0.3177489177489177
r2d 0.3111888111888112
d2u 0.3128957014328557
r2u 0.3078973675441519
u2d 0.24625124958347216
u2r 0.2745751416194602
All 0.2901354932253387


In [10]:
def counting_offensiveness_by_subreddits(interactions,subreddit,limit=3000):
    Counting={'Republican':Counter(),'Democrat':Counter(),'Unknown':Counter()}
    a=0
    for idx,(comment_id,line) in enumerate(interactions.iterrows()):
        #print(line['subreddit'],subreddit)
        if line['subreddit']==subreddit:
            text_id,mask=prepare_features(line['text'])
            text_id=text_id.unsqueeze(0).cuda()
            mask=mask.unsqueeze(0).cuda()
            output = model.forward(text_id,attention_mask=mask)[0]
            _, predicted = torch.max(output.data, 1)
            Counting[line['politics']][predicted.item()]+=1
            a+=1
            if a==limit:
                break
    return Counting
        

In [11]:
#key_word=['conservative','conspiracy','liberal','political','politics','fascism','socialism','donald','WayOfTheBern','bernie']
key_word=['conservative','democrat',]
political_subreddit=[]
all_sub={''}
for key in comment_lists:
    all_sub.update(set(list(comment_lists[key]['subreddit'])))

for sub in all_sub:
    for key in key_word:
        if key in sub.lower():
            political_subreddit.append(sub)
            #print(sub)
political_subreddit=['Conservative','democrats']

In [14]:
subreddit_counting={}
for sub in political_subreddit:
    print(sub)
    subreddit_counting[sub]=counting_offensiveness_by_subreddits(df_comments,sub)

conservatives
DemocratsforDiversity
askaconservative
DemocraticSocialism
ConservativeMemes
justicedemocrats
democrats
Conservative
conservativecartoons
ConservativesOnly
Forum_Democratie
AskDemocrats
democraticparty
AskConservatives


In [16]:
print(subreddit_counting)

{'conservatives': {'Republican': Counter({0: 73, 1: 22}), 'Democrat': Counter({0: 17, 1: 6}), 'Unknown': Counter({0: 17, 1: 8})}, 'DemocratsforDiversity': {'Republican': Counter({0: 207, 1: 62}), 'Democrat': Counter({0: 616, 1: 167}), 'Unknown': Counter({0: 57, 1: 24})}, 'askaconservative': {'Republican': Counter({0: 68, 1: 40}), 'Democrat': Counter({0: 60, 1: 38}), 'Unknown': Counter({0: 17, 1: 11})}, 'DemocraticSocialism': {'Republican': Counter({0: 7, 1: 3}), 'Democrat': Counter({0: 50, 1: 17}), 'Unknown': Counter({0: 14, 1: 5})}, 'ConservativeMemes': {'Republican': Counter({0: 1}), 'Democrat': Counter({1: 2, 0: 2}), 'Unknown': Counter({0: 2})}, 'justicedemocrats': {'Republican': Counter(), 'Democrat': Counter({0: 6, 1: 2}), 'Unknown': Counter({0: 1})}, 'democrats': {'Republican': Counter({0: 44, 1: 21}), 'Democrat': Counter({0: 76, 1: 29}), 'Unknown': Counter({0: 17, 1: 6})}, 'Conservative': {'Republican': Counter({0: 509, 1: 254}), 'Democrat': Counter({0: 279, 1: 129}), 'Unknown':

In [18]:
for key in subreddit_counting:
    print("In subreddit",key,":")
    counting=subreddit_counting[key]
    try:
        print("Republican:",counting['Republican'][1]/(counting['Republican'][0]+counting['Republican'][1]),
         "Democrat:",counting['Democrat'][1]/(counting['Democrat'][0]+counting['Democrat'][1]),
         "Unknown:",counting['Unknown'][1]/(counting['Unknown'][0]+counting['Unknown'][1]))
    except:
        print("divided by zero!")

In subreddit conservatives :
Republican: 0.23157894736842105 Democrat: 0.2608695652173913 Unknown: 0.32
In subreddit DemocratsforDiversity :
Republican: 0.23048327137546468 Democrat: 0.21328224776500637 Unknown: 0.2962962962962963
In subreddit askaconservative :
Republican: 0.37037037037037035 Democrat: 0.3877551020408163 Unknown: 0.39285714285714285
In subreddit DemocraticSocialism :
Republican: 0.3 Democrat: 0.2537313432835821 Unknown: 0.2631578947368421
In subreddit ConservativeMemes :
Republican: 0.0 Democrat: 0.5 Unknown: 0.0
In subreddit justicedemocrats :
divided by zero!
In subreddit democrats :
Republican: 0.3230769230769231 Democrat: 0.2761904761904762 Unknown: 0.2608695652173913
In subreddit Conservative :
Republican: 0.3328964613368283 Democrat: 0.3161764705882353 Unknown: 0.27586206896551724
In subreddit conservativecartoons :
Republican: 0.0 Democrat: 0.0 Unknown: 0.0
In subreddit ConservativesOnly :
Republican: 0.42857142857142855 Democrat: 0.0 Unknown: 0.0
In subreddit 

# Find Offensiveness of Subreddits (interactions or not?)

In [17]:
def print_interaction_scores(just2_interaction_counting):
    for key in just2_interaction_counting:
        print("In subreddit",key,":")
        counting=just2_interaction_counting[key]
        print("Republican:", 0 if (counting['Republican'][0]+counting['Republican'][1])==0 else counting['Republican'][1]/(counting['Republican'][0]+counting['Republican'][1]),
             "Democrat:",0 if (counting['Democrat'][0]+counting['Democrat'][1])==0 else counting['Democrat'][1]/(counting['Democrat'][0]+counting['Democrat'][1]),
#              "Unknown:",0 if (counting['Unknown'][0]+counting['Unknown'][1])==0 else counting['Unknown'][1]/(counting['Unknown'][0]+counting['Unknown'][1]))


In [22]:
def go_through_all_comments(list_of_subreddits):
    counting={}
    for sub in list_of_subreddits:
        print(sub)  
        counting[sub]=counting_offensiveness_by_subreddits(df_comments,sub)
    print(counting)
    print_interaction_scores(counting)

In [31]:
def go_through_interactions(list_of_subreddits):
    for interaction in comment_lists:
        print(interaction)
        interaction_counting={}
        for sub in list_of_subreddits:
            #print(sub)  
            interaction_counting[sub]=counting_offensiveness_by_subreddits(comment_lists[interaction],sub)
        print(interaction_counting)
        print_interaction_scores(interaction_counting)

In [12]:
go_through_all_comments(political_subreddit)
# just2_counting={}
# for sub in political_subreddit:
#     print(sub)  
#     just2_counting[sub]=counting_offensiveness_by_subreddits(df_comments,sub)
# print(just2_counting)
# print_interaction_scores(just2_counting)

Conservative
democrats
{'Conservative': {'Republican': Counter({0: 1049, 1: 532}), 'Democrat': Counter({0: 601, 1: 301}), 'Unknown': Counter({0: 367, 1: 150})}, 'democrats': {'Republican': Counter({0: 508, 1: 271}), 'Democrat': Counter({0: 1271, 1: 538}), 'Unknown': Counter({0: 299, 1: 113})}}


NameError: name 'print_interaction_scores' is not defined

In subreddit Conservative :
Republican: 0.33649588867805186 Democrat: 0.33370288248337027 Unknown: 0.2901353965183752
In subreddit democrats :
Republican: 0.3478818998716303 Democrat: 0.2974018794914317 Unknown: 0.27427184466019416


In [33]:
go_through_interactions(political_subreddit)

d2d
{'Conservative': {'Republican': Counter(), 'Democrat': Counter({0: 91, 1: 71}), 'Unknown': Counter()}, 'democrats': {'Republican': Counter(), 'Democrat': Counter({0: 101, 1: 37}), 'Unknown': Counter()}}
In subreddit Conservative :
Republican: 0 Democrat: 0.4382716049382716 Unknown: 0
In subreddit democrats :
Republican: 0 Democrat: 0.26811594202898553 Unknown: 0
r2r
{'Conservative': {'Republican': Counter({0: 316, 1: 144}), 'Democrat': Counter(), 'Unknown': Counter()}, 'democrats': {'Republican': Counter({0: 15, 1: 10}), 'Democrat': Counter(), 'Unknown': Counter()}}
In subreddit Conservative :
Republican: 0.3130434782608696 Democrat: 0 Unknown: 0
In subreddit democrats :
Republican: 0.4 Democrat: 0 Unknown: 0
d2r
{'Conservative': {'Republican': Counter(), 'Democrat': Counter({0: 188, 1: 120}), 'Unknown': Counter()}, 'democrats': {'Republican': Counter(), 'Democrat': Counter({0: 39, 1: 20}), 'Unknown': Counter()}}
In subreddit Conservative :
Republican: 0 Democrat: 0.389610389610389

In [38]:
'Jokes' in all_sub

True

In [24]:
cute_reddit=['cats','dogs']

In [25]:
go_through_all_comments(cute_reddit)

cats
dogs
{'cats': {'Republican': Counter({0: 387, 1: 67}), 'Democrat': Counter({0: 1271, 1: 209}), 'Unknown': Counter({0: 347, 1: 50})}, 'dogs': {'Republican': Counter({0: 120, 1: 47}), 'Democrat': Counter({0: 782, 1: 252}), 'Unknown': Counter({0: 246, 1: 62})}}
In subreddit cats :
Republican: 0.14757709251101322 Democrat: 0.14121621621621622 Unknown: 0.12594458438287154
In subreddit dogs :
Republican: 0.281437125748503 Democrat: 0.2437137330754352 Unknown: 0.2012987012987013


In [21]:
misc=['funny','science','gaming','food']

In [23]:
go_through_all_comments(misc)

funny
science
gaming
food
{'funny': {'Republican': Counter({0: 466, 1: 182}), 'Democrat': Counter({0: 1253, 1: 460}), 'Unknown': Counter({0: 477, 1: 162})}, 'science': {'Republican': Counter({0: 488, 1: 171}), 'Democrat': Counter({0: 1164, 1: 380}), 'Unknown': Counter({0: 594, 1: 203})}, 'gaming': {'Republican': Counter({0: 495, 1: 170}), 'Democrat': Counter({0: 1277, 1: 422}), 'Unknown': Counter({0: 483, 1: 153})}, 'food': {'Republican': Counter({0: 457, 1: 91}), 'Democrat': Counter({0: 1497, 1: 304}), 'Unknown': Counter({0: 556, 1: 95})}}
In subreddit funny :
Republican: 0.2808641975308642 Democrat: 0.2685347343841214 Unknown: 0.2535211267605634
In subreddit science :
Republican: 0.25948406676783003 Democrat: 0.24611398963730569 Unknown: 0.2547051442910916
In subreddit gaming :
Republican: 0.2556390977443609 Democrat: 0.24838140082401414 Unknown: 0.24056603773584906
In subreddit food :
Republican: 0.16605839416058393 Democrat: 0.1687951138256524 Unknown: 0.14592933947772657


In [32]:
go_through_interactions(misc)
# for interaction in comment_lists:
#     print(interaction)
#     just2_interaction_counting={}
#     for sub in normal_reddit:
#         #print(sub)  
#         just2_interaction_counting[sub]=counting_offensiveness_by_subreddits(comment_lists[interaction],sub)
#     print(just2_interaction_counting)
#     print_interaction_scores(just2_interaction_counting)

d2d
{'funny': {'Republican': Counter(), 'Democrat': Counter({0: 342, 1: 139}), 'Unknown': Counter()}, 'science': {'Republican': Counter(), 'Democrat': Counter({0: 161, 1: 69}), 'Unknown': Counter()}, 'gaming': {'Republican': Counter(), 'Democrat': Counter({0: 203, 1: 60}), 'Unknown': Counter()}, 'food': {'Republican': Counter(), 'Democrat': Counter({0: 10, 1: 2}), 'Unknown': Counter()}}
In subreddit funny :
Republican: 0 Democrat: 0.288981288981289 Unknown: 0
In subreddit science :
Republican: 0 Democrat: 0.3 Unknown: 0
In subreddit gaming :
Republican: 0 Democrat: 0.22813688212927757 Unknown: 0
In subreddit food :
Republican: 0 Democrat: 0.16666666666666666 Unknown: 0
r2r
{'funny': {'Republican': Counter({0: 57, 1: 38}), 'Democrat': Counter(), 'Unknown': Counter()}, 'science': {'Republican': Counter({0: 33, 1: 17}), 'Democrat': Counter(), 'Unknown': Counter()}, 'gaming': {'Republican': Counter({0: 31, 1: 10}), 'Democrat': Counter(), 'Unknown': Counter()}, 'food': {'Republican': Counte

In [39]:
misc2=['announcements','Jokes','news','sports']

In [40]:
go_through_all_comments(misc2)

announcements
Jokes
news
sports
{'announcements': {'Republican': Counter({0: 106, 1: 32}), 'Democrat': Counter({0: 287, 1: 168}), 'Unknown': Counter({0: 141, 1: 75})}, 'Jokes': {'Republican': Counter({0: 668, 1: 193}), 'Democrat': Counter({0: 1100, 1: 364}), 'Unknown': Counter({0: 521, 1: 154})}, 'news': {'Republican': Counter({0: 334, 1: 200}), 'Democrat': Counter({0: 1057, 1: 614}), 'Unknown': Counter({0: 492, 1: 303})}, 'sports': {'Republican': Counter({0: 508, 1: 191}), 'Democrat': Counter({0: 1076, 1: 448}), 'Unknown': Counter({0: 569, 1: 208})}}
In subreddit announcements :
Republican: 0.2318840579710145 Democrat: 0.36923076923076925 Unknown: 0.3472222222222222
In subreddit Jokes :
Republican: 0.22415795586527293 Democrat: 0.24863387978142076 Unknown: 0.22814814814814816
In subreddit news :
Republican: 0.37453183520599254 Democrat: 0.36744464392579296 Unknown: 0.38113207547169814
In subreddit sports :
Republican: 0.2732474964234621 Democrat: 0.29396325459317585 Unknown: 0.2676962

In [ ]:
go_through_interactions(misc2)

d2d
{'announcements': {'Republican': Counter(), 'Democrat': Counter({0: 20, 1: 14}), 'Unknown': Counter()}, 'Jokes': {'Republican': Counter(), 'Democrat': Counter({0: 31, 1: 10}), 'Unknown': Counter()}, 'news': {'Republican': Counter(), 'Democrat': Counter({0: 1850, 1: 1150}), 'Unknown': Counter()}, 'sports': {'Republican': Counter(), 'Democrat': Counter({0: 34, 1: 11}), 'Unknown': Counter()}}
In subreddit announcements :
Republican: 0 Democrat: 0.4117647058823529 Unknown: 0
In subreddit Jokes :
Republican: 0 Democrat: 0.24390243902439024 Unknown: 0
In subreddit news :
Republican: 0 Democrat: 0.38333333333333336 Unknown: 0
In subreddit sports :
Republican: 0 Democrat: 0.24444444444444444 Unknown: 0
r2r
{'announcements': {'Republican': Counter({1: 2}), 'Democrat': Counter(), 'Unknown': Counter()}, 'Jokes': {'Republican': Counter({0: 18, 1: 3}), 'Democrat': Counter(), 'Unknown': Counter()}, 'news': {'Republican': Counter({0: 358, 1: 200}), 'Democrat': Counter(), 'Unknown': Counter()}, 's

In [ ]:
misc3=['books','Art','gonewild']
go_through_all_comments(misc3)